In [ ]:
# Import and create dfs.
import pandas as pd
import numpy as np
import re

file_assetList = 'Input/IEA equipment inventory_20190925.xlsx'
df_input = pd.read_excel(file_assetList, sheet_name='Sheet1')
#display(df_input.columns)

In [ ]:
#display(df_input)

In [ ]:
#Extract Floor PDU Circuits

#Check for non-conforming Floor PDU Circuit names (e.g. Not PDU-01-6L1)
print('The following are non-conforming Floor PDU Circuit names, e.g. NOT PDU-1-6L1/23')
display(df_input.loc[~df_input['PDU No. / Socket No.'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'PDU No. / Socket No.'])
display(df_input.loc[~df_input['PDU No. / Socket No..1'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'PDU No. / Socket No..1'])
display(df_input.loc[~df_input['PDU No. / Socket No..2'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'PDU No. / Socket No..2'])
display(df_input.loc[~df_input['PDU No. / Socket No..3'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'PDU No. / Socket No..3'])
display(df_input.loc[~df_input['Remarks'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'Remarks'])


df_pdus = df_input['PDU No. / Socket No.'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus[0].sort_values().dropna().unique()

df_pdus1 = df_input['PDU No. / Socket No..1'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus1[0].sort_values().dropna().unique()

df_pdus2 = df_input['PDU No. / Socket No..2'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus2[0].sort_values().dropna().unique()

df_pdus3 = df_input['PDU No. / Socket No..3'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus3[0].sort_values().dropna().unique()

df_pdus4 = df_input['Remarks'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus4[0].sort_values().dropna().unique()

list_pdus = np.append(df_pdus[0].sort_values().dropna().unique(), df_pdus1[0].sort_values().dropna().unique())
list_pdus = np.append(list_pdus, df_pdus2[0].sort_values().dropna().unique())
list_pdus = np.append(list_pdus, df_pdus3[0].sort_values().dropna().unique())
list_pdus = np.append(list_pdus, df_pdus4[0].sort_values().dropna().unique())

#df_racks = pd.DataFrame(df_input['IEA Rack'].sort_values().dropna().unique(), columns=['Name'])

df_pdus_final = pd.DataFrame(list_pdus, columns=['Name'])
df_circuits = pd.DataFrame(df_pdus_final['Name'].sort_values().unique(), columns=['Circuit'])
display(df_circuits)

df_circuits.to_csv("Output/Circuits.csv", ",", index=False)

In [ ]:
#Create "Add Rack PDUs" Import Sheet

#Extracting circuit info to create standard circuits
df_circuits['Panel'] = df_circuits['Circuit'].str.extract('PDU-(\d+)-.*',  flags=re.IGNORECASE)
df_circuits['Panel'] = df_circuits['Panel'].fillna(0).astype(int)
df_circuits['Line'] = df_circuits['Circuit'].str.extract('PDU-\d+-(\d+).*',  flags=re.IGNORECASE)
df_circuits['Line'] = df_circuits['Line'].fillna(0).astype(int)
df_circuits['Phase'] = df_circuits['Circuit'].str.extract('PDU-\d+-\d+L(\d+)',  flags=re.IGNORECASE)
df_circuits['Phase'] = df_circuits['Phase'].fillna(0).astype(int)

df_circuits['Circuit Name'] = "PDU-" + df_circuits['Panel'].astype(str) + "-" + df_circuits['Line'].astype(str) + "L" + df_circuits['Phase'].astype(str)
display(df_circuits['Circuit Name'].sort_values().dropna().unique())

df_rpdus = pd.DataFrame(df_circuits['Circuit Name'].sort_values().dropna().unique(), columns=['Name'])
df_rpdus['PDU Name'] = ""
df_rpdus['Make'] = "Raritan"
df_rpdus['Model'] = ""
df_rpdus['Cabinet'] = ""
df_rpdus['Depth Position'] = ""
df_rpdus['Cabinet Side'] = ""
df_rpdus['U Position'] = ""
df_rpdus['IP Address'] = ""
df_rpdus['Proxy ID'] = ""
df_rpdus.to_csv("Output/Circuits_and_RackPDU.csv", ",", index=False)

In [ ]:
#Extract Rack from df_input
#Create Add Cabinet import sheet
print('List of racks', df_input['IEA Rack'].sort_values().dropna().unique())
df_racks = pd.DataFrame(df_input['IEA Rack'].sort_values().dropna().unique(), columns=['Name'])
df_racks['Make'] = ''
df_racks['Model'] = ''
df_racks['Row Label'] = df_racks['Name'].str.extract('\d(\w)-\d+')
df_racks['Position in Row'] = df_racks['Name'].str.extract('\d\w-[0]*(\d+)')
df_racks['Status'] = 'Planned'
df_racks['Capacity(kW)'] = 6
display(df_racks)
df_racks.to_csv('Output/Cabinets.csv', ',', index=False)

In [ ]:
df_racks